<a href="https://colab.research.google.com/github/Arul-Vincy-Mary/nerd/blob/main/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import required libraries
import os
import csv
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from numpy import mean, sqrt, square

In [2]:
#current working directory
os.getcwd()

'/content'

In [3]:
#changing the working directory
os.chdir('/content/drive/MyDrive/gestures-dataset')

In [4]:
#list the directory
os.listdir()

['U01', 'U02', 'U03', 'U04', 'U05', 'U06', 'U07', 'U08']

In [5]:
#statistical features of the data
def features(data):
  column = data.columns
  lists = []

  for i in column:
    mean = data[i].mean()
    lists.append(mean)
    median = data[i].median()
    lists.append(median)
    rms_value = np.sqrt(np.mean(np.square(data[i])))
    lists.append(rms_value)
    var = data[i].var()
    lists.append(var)
    std = data[i].std()
    lists.append(std)
    kurt = data[i].kurtosis()
    lists.append(kurt)
    skew = data[i].skew()
    lists.append(skew)
  return lists

In [6]:
#flatten 
def flatten(values):
  lists_1 = []
  for sub_list in values:
    for item in sub_list:
      lists_1.append(item)
  return lists_1 

In [7]:
#sampling
def sample(data):
  data_1 = data.sample(n=10, replace=True)
  col = data.columns
  val = []
  for i in col:
   q = data_1[i]. tolist()
   val.append(q)
  val = flatten(val)
  return val  

In [ ]:
data_1 = pd.DataFrame(columns=['C' +str(x) for x in range(1,53)])
home_path= os.getcwd()
directory= os.listdir()
for dir in directory:
  path = home_path + '/'+ dir
  os.chdir(home_path + '/' + dir)
  directory_1 = os.listdir(path)
  for sub_directory in directory_1:
    path_1 = home_path + '/' +dir+ '/' + sub_directory
    os.chdir(home_path + '/' +dir+ '/' + sub_directory)
    path_1 = os.listdir(path_1)
    for file_name in path_1:
      file_path = print(home_path + '/' +dir+ '/' + sub_directory+ '/' +file_name)
      ges_data = []
      data = pd.read_csv(file_name, sep = " ")
      data.columns = ['time_stamp1','time_stamp2','time_stamp3','roll','pitch','yaw']
      data = data.drop(['time_stamp1','time_stamp2','time_stamp3'], axis= 1)
      features_1 = features(data)
      sample_1 = sample(data)
      ges_data = features_1 + sample_1+[sub_directory]
      df= pd.DataFrame([ges_data], columns=['C' +str(x) for x in range(1,53)])
      data_1 = pd.concat([data_1,df])

In [9]:
#rest the index
new_data = data_1.reset_index(drop=True)

In [10]:
#copy of the dataste
gesture = new_data.copy(deep=True)

In [11]:
#shape of the dataset
gesture.shape

(3251, 52)

In [12]:
gesture.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52
0,1.426207,0.612916,2.613889,5.198213,2.279959,1.414591,1.564126,-1.449781,-1.072602,3.211065,8.893160,2.982140,1.528427,-0.996622,9.853797,9.806650,9.875106,0.455435,0.674859,-1.229070,0.178404,-0.153229,-0.153229,-0.459687,-0.153229,-0.306458,0.000000,-0.153229,-0.459687,6.435614,1.838747,-1.072602,-1.685518,-0.153229,-1.072602,-1.225831,-2.145205,-1.072602,-0.153229,1.532289,-0.153229,10.726024,9.653421,10.419566,10.726024,9.806650,9.653421,10.726024,10.419566,9.193734,10.726024,01
1,1.131537,0.459687,2.441327,5.069679,2.251595,0.340229,0.371441,-1.496929,-1.072602,3.055430,7.686094,2.772381,3.684154,-1.531194,9.983453,10.113108,9.999693,0.351584,0.592945,1.529214,-1.014633,-0.153229,0.000000,-0.153229,5.209783,1.532289,5.209783,0.000000,1.532289,1.685518,-3.064578,-3.064578,-1.072602,-3.064578,1.991976,0.153229,1.991976,-0.919373,0.153229,-8.887277,1.072602,9.346964,10.266337,9.346964,10.266337,10.266337,10.266337,10.572795,10.266337,8.580819,9.500193,01
2,1.720879,1.225831,2.789375,5.220789,2.284905,0.506976,1.262921,-1.437995,-1.072602,3.361916,10.004204,3.162942,1.421851,0.031414,9.441258,9.806650,9.531414,1.853043,1.361265,9.076539,-2.837289,2.604892,-0.153229,-0.153229,6.282385,0.153229,1.225831,-0.153229,2.604892,-0.153229,1.225831,1.838747,-3.830723,-1.072602,-1.532289,-0.766145,-7.661446,-1.072602,1.838747,-3.830723,-7.661446,10.572795,8.887277,9.959879,9.040505,9.806650,9.653421,9.959879,10.572795,8.887277,9.653421,01
3,1.119750,-0.153229,2.447608,5.131689,2.265323,0.071415,1.136033,-1.520502,-1.225831,2.646165,5.081117,2.254133,1.243180,-0.828329,9.783076,9.959879,9.823763,0.864211,0.929630,4.857251,-1.818821,-0.766145,1.838747,-0.459687,2.604892,-0.459687,-0.459687,5.363012,2.604892,0.459687,-0.459687,-0.306458,-1.225831,-0.766145,0.919373,-4.443638,-0.766145,2.145205,0.919373,-6.588843,-4.443638,9.653421,9.959879,10.419566,9.653421,9.193734,10.419566,10.419566,9.653421,7.201759,9.193734,01
4,1.400950,0.536301,2.231047,3.246824,1.801894,-0.319514,0.649635,-1.061657,-0.536301,2.625093,6.207383,2.491462,3.517154,-1.581110,9.937989,9.883264,9.957185,0.411271,0.641304,0.857492,0.256622,0.153229,0.000000,0.459687,0.000000,2.911349,2.911349,0.459687,0.153229,2.911349,4.290410,-1.072602,-0.459687,-0.612916,-2.758120,-7.814674,1.838747,-1.225831,-1.072602,1.838747,0.459687,10.419566,10.419566,9.806650,9.346964,9.346964,9.346964,9.806650,10.419566,9.346964,10.113108,01


Model building:

splitting data as test and train:

In [13]:
# Splitting data as test and train
from sklearn.model_selection import train_test_split
gesture_columns_list = list(gesture.columns)
features = list(set(gesture_columns_list)-set(['C52']))

x = gesture[features].values
y = gesture['C52'].values

x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.20)

In [14]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2600, 51)
(651, 51)
(2600,)
(651,)


In [15]:
# Standardising all the variables 
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
x_train = scalar.fit_transform(x_train)
x_test = scalar.transform(x_test)

In [19]:
#importing libraries for knn,RF,SVM
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

KNN:

In [17]:
# Setting the K value
kclassifier = KNeighborsClassifier(n_neighbors = 3)

# Fitting the values of x and y
kclassifier.fit(x_train, y_train)

# Predicting 
prediction = kclassifier.predict(x_test)

# Accuracy Score
accuracy_score = accuracy_score(prediction, y_test)
print(accuracy_score)

0.5099846390168971


2.Random Forest:

In [35]:
# Building Random Forest model
rf = RandomForestClassifier(n_estimators= 200, criterion='gini' )
rf.fit(x_train, y_train)

# Predicting the model on test data
predictions_rf = rf.predict(x_test)

#accuracy score
print(accuracy_score(y_test, predictions_rf))

0.8433179723502304


3.Support Vector Machine:

In [57]:
#Building SVM model
svm_clf = SVC(gamma='auto',C = 10, kernel = 'rbf', random_state=None)
svm_clf.fit(x_train, y_train) 

#predicting the model on the test data
y_pred = svm_clf.predict(x_test)

#accuracy score
print(accuracy_score(y_test, y_pred))

0.7373271889400922


Hyper parameter tunning:

i) SVM:

In [67]:
from sklearn.model_selection import GridSearchCV

In [53]:
param_grid = { 'gamma': [0.01,0.1,1],
                 'C' :[0.1,1,10],
                 'kernel':['rbf','linear'],
                }

In [54]:
grid_search = GridSearchCV(estimator= svm_clf, param_grid= param_grid, cv= 10,n_jobs = -1, verbose = False)

In [55]:
grid_search.fit(x_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=10, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1],
                         'kernel': ['rbf', 'linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=False)

In [58]:
grid_search.best_params_

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

In [56]:
grid_pred = grid_search.predict(x_test)
print(accuracy_score(y_test, grid_pred))

0.7450076804915514


ii) Random Forest:

In [59]:
param_grid_1 = {'bootstrap': [True],
                'max_depth': [50, 100, 150],
                'max_features': [2, 3, 5],
                'min_samples_leaf': [3, 4, 5],
                'min_samples_split': [8, 10, 20],
                'n_estimators': [200, 500, 1000]
                }

In [60]:
grid_search_2 = GridSearchCV(estimator = rf, param_grid = param_grid_1, cv = 3, n_jobs = -1, verbose = False)

In [61]:
grid_search_3.fit(x_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=200, n_jobs=None,
                                              oob_score=False,
                                              rando

In [62]:
grid_search_3.best_params_

{'bootstrap': True,
 'max_depth': 150,
 'max_features': 5,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 1000}

In [63]:
me = grid_search_3.best_estimator_

In [64]:
grid_pred_2 = me.predict(x_test)
print(accuracy_score(y_test, grid_pred_2))

0.8172043010752689


with PCA:

In [ ]:
input_columns = list(data_2.iloc[:,:51].columns)

input_data = data_2[input_columns]
input_data.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51
0,1.426207,0.612916,2.613889,5.198213,2.279959,1.414591,1.564126,-1.449781,-1.072602,3.211065,8.893160,2.982140,1.528427,-0.996622,9.853797,9.806650,9.875106,0.455435,0.674859,-1.229070,0.178404,-0.306458,1.838747,1.838747,0.306458,-0.153229,6.435614,-0.153229,0.919373,0.612916,1.838747,-1.225831,-0.153229,-0.153229,-0.306458,-1.685518,1.532289,-1.072602,-1.532289,-6.588843,-0.153229,9.806650,10.726024,10.726024,9.959879,9.653421,9.193734,10.726024,9.040505,8.887277,10.726024
0,1.131537,0.459687,2.441327,5.069679,2.251595,0.340229,0.371441,-1.496929,-1.072602,3.055430,7.686094,2.772381,3.684154,-1.531194,9.983453,10.113108,9.999693,0.351584,0.592945,1.529214,-1.014633,1.532289,-3.064578,-3.064578,1.532289,4.750096,5.209783,-0.153229,1.685518,0.459687,1.685518,0.153229,1.072602,1.072602,0.153229,1.225831,1.991976,-3.064578,-8.887277,-1.532289,-8.887277,10.266337,9.500193,9.500193,10.266337,10.266337,10.266337,9.346964,8.580819,9.959879,8.580819
0,1.720879,1.225831,2.789375,5.220789,2.284905,0.506976,1.262921,-1.437995,-1.072602,3.361916,10.004204,3.162942,1.421851,0.031414,9.441258,9.806650,9.531414,1.853043,1.361265,9.076539,-2.837289,1.225831,0.153229,0.153229,0.153229,0.153229,0.153229,6.129157,1.225831,-0.153229,-0.153229,-7.661446,-0.766145,-0.766145,-0.766145,-0.766145,-0.766145,5.209783,0.000000,-1.838747,-3.830723,9.653421,9.959879,9.806650,9.959879,9.959879,9.806650,10.419566,10.113108,9.500193,8.887277
0,1.119750,-0.153229,2.447608,5.131689,2.265323,0.071415,1.136033,-1.520502,-1.225831,2.646165,5.081117,2.254133,1.243180,-0.828329,9.783076,9.959879,9.823763,0.864211,0.929630,4.857251,-1.818821,0.459687,2.451663,2.604892,-0.766145,-0.153229,1.838747,-0.766145,-0.459687,2.604892,-0.919373,-6.588843,-3.524265,0.919373,-0.306458,-0.766145,-1.225831,-0.306458,-1.379060,0.919373,-0.766145,7.201759,9.193734,9.653421,9.653421,9.959879,9.959879,9.653421,10.419566,9.653421,9.959879
0,1.400950,0.536301,2.231047,3.246824,1.801894,-0.319514,0.649635,-1.061657,-0.536301,2.625093,6.207383,2.491462,3.517154,-1.581110,9.937989,9.883264,9.957185,0.411271,0.641304,0.857492,0.256622,1.838747,0.612916,1.838747,0.459687,4.290410,0.000000,4.903325,0.000000,0.153229,4.290410,0.000000,-3.830723,0.000000,-0.612916,0.459687,-0.459687,-0.459687,-0.459687,-1.072602,0.459687,10.419566,8.734048,10.419566,9.806650,10.113108,10.419566,11.338940,10.419566,10.419566,10.113108


In [ ]:
# Scaling data using (x-mu)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_std = True)
input_data = scaler.fit_transform(input_data)

In [ ]:
input_data = pd.DataFrame(input_data,columns=input_columns)
input_data.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51
0,-2.041552,-2.393595,-3.709861,-22.110934,-2.522380,1.437922,1.375416,-0.056932,0.130912,-1.096014,-8.776824,-0.915799,0.785012,-0.852419,1.938430,2.052753,0.682439,-22.552296,-3.566251,-2.495473,-0.077487,-3.632906,-1.514520,-1.718605,-3.245568,-3.601657,3.062504,-3.597180,-2.570485,-2.811899,-1.800758,0.073528,1.140238,1.297002,1.238274,-0.442389,2.830705,0.256167,-0.236701,-5.214732,1.290073,1.920852,2.810108,2.918844,2.135354,1.731001,1.225595,2.721970,1.016278,0.990591,2.727249
1,-2.336223,-2.546824,-3.882423,-22.239467,-2.550744,0.363561,0.182731,-0.104080,0.130912,-1.251649,-9.983890,-1.125558,2.940739,-1.386990,2.068085,2.359211,0.807026,-22.656146,-3.648164,0.262811,-1.270523,-1.794159,-6.417845,-6.621930,-2.019737,1.301668,1.836673,-3.597180,-1.804340,-2.965128,-1.953987,1.452588,2.366069,2.522833,1.697961,2.468960,3.290392,-1.735809,-7.591689,-0.158178,-7.443975,2.380539,1.584277,1.693013,2.441812,2.343917,2.298198,1.342910,0.556592,2.063193,0.582044
2,-1.746881,-1.780680,-3.534376,-22.088357,-2.517435,0.530308,1.074210,-0.045146,0.130912,-0.945163,-7.665779,-0.734996,0.678436,0.175617,1.525891,2.052753,0.338747,-21.154688,-2.879845,7.810136,-3.093180,-2.100617,-3.200038,-3.404123,-3.398797,-3.295199,-3.219881,2.685206,-2.264027,-3.578044,-3.792734,-6.362087,0.527322,0.684086,0.778587,0.476984,0.532271,6.538552,1.295588,-0.464636,-2.387421,1.767623,2.043963,1.999470,2.135354,2.037459,1.838511,2.415512,2.088881,1.603507,0.888502
3,-2.348010,-3.159740,-3.876143,-22.177457,-2.537016,0.094746,0.947322,-0.127653,-0.022317,-1.660914,-12.588866,-1.643805,0.499765,-0.684126,1.867709,2.205982,0.631095,-22.143519,-3.311480,3.590848,-2.074712,-2.866761,-0.901604,-0.952460,-4.318171,-3.601657,-1.534363,-4.210096,-3.949545,-0.819923,-4.558878,-5.289484,-2.230798,2.369604,1.238274,0.476984,0.072585,1.022311,-0.083472,2.293484,0.677157,-0.684039,1.277818,1.846241,1.828896,2.037459,1.991740,1.649367,2.395339,1.756735,1.961104
4,-2.066810,-2.470209,-4.092704,-24.062322,-3.000445,-0.296182,0.460925,0.331192,0.667212,-1.681985,-11.462600,-1.406477,2.773739,-1.436906,2.022622,2.129368,0.764517,-22.596459,-3.599805,-0.408911,0.000732,-1.487701,-2.740351,-1.718605,-3.092339,0.841982,-3.373110,1.459374,-3.489858,-3.271586,0.650905,1.299359,-2.537256,1.450231,0.931816,1.702816,0.838729,0.869082,0.835901,0.301509,1.902989,2.533768,0.818132,2.612386,1.982125,2.190688,2.451427,3.334886,2.395339,2.522880,2.114333


In [ ]:
u, s, v = np.linalg.svd(input_data)  #decomposing using SVD
gest = data_2[input_columns].dot(v.T)
gest.columns= ['col' +str(x) for x in range(1,52)]
gest

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,col51
0,2.981757,0.930167,-6.504309,4.145298,17.914827,11.796062,-3.279976,-10.262140,-12.113334,10.561944,16.700171,-2.743487,4.656042,4.769775,-4.336217,-1.164041,2.694489,0.190694,-1.102917,-1.636381,2.623121,-3.800252,2.892837,0.285310,-0.701675,3.327975,2.191827,3.191380,-1.443529,2.688537,0.482867,-3.332678,-1.434384,4.333419,-2.708510,5.952898,-6.081190,-1.518520,2.555949,5.193041,4.841290,0.944825,-0.324398,2.624692,-3.186598,-2.284424,-3.435495,0.881866,0.091341,2.003784,1.404924
0,2.562786,1.175193,-5.584769,4.382689,17.248264,15.854818,-0.778055,-9.443510,-14.761620,10.309589,12.466334,-3.622135,7.322218,3.627075,0.518161,-1.184425,2.747804,-3.839478,-4.190719,-1.342398,-4.194872,-4.409375,1.369324,2.529531,-2.623324,-0.844974,1.695318,2.735177,-1.549703,4.081940,0.879898,5.294059,-3.694276,2.422979,-5.307708,5.512484,-5.911205,-0.167642,1.837931,6.409856,4.369213,0.976204,1.008440,1.968526,-3.486103,-1.683872,-4.133137,0.620888,-0.086144,2.085640,1.412180
0,4.421553,1.876080,-7.486937,5.805088,18.657680,11.849551,-2.497996,-11.491260,-14.337524,8.133908,15.353551,-2.411461,1.740058,2.184639,-1.008837,-1.176413,-0.570894,0.165828,-0.643748,0.147326,1.178720,1.860963,3.000123,0.231278,3.299346,3.690817,-4.877524,-0.784119,0.138205,0.823937,5.865086,-0.138524,-4.104444,0.272523,-9.059541,0.037832,-8.928936,-1.120540,2.980539,6.898731,3.088851,-0.470193,2.013683,2.512737,-2.413246,-2.092192,-4.502322,1.458307,-0.124726,1.963769,0.982767
0,2.478851,1.608154,-2.689707,5.834392,16.605050,12.700200,-4.093186,-10.220596,-11.758723,9.639325,15.448624,-2.770145,1.617935,5.129490,-4.053044,0.558379,-1.470494,0.423045,-1.049579,-1.543352,0.448407,2.016325,3.017876,-2.825164,-0.181563,0.708133,-4.293064,-0.412121,-3.681796,1.566023,-0.945304,-0.871491,-1.350733,1.559220,-5.684127,2.397597,-7.981382,-0.992044,2.174206,6.666250,3.631261,0.004376,1.089513,2.001774,-2.874913,-1.983045,-4.211922,0.996139,-0.214402,2.144631,1.184338
0,1.230209,2.857414,-4.377300,3.648867,19.568313,10.680346,-1.365403,-14.687043,-12.609694,10.118383,15.504839,-2.052668,4.265440,1.457049,-1.615563,0.056012,1.230966,0.048883,-1.306581,-0.910279,0.422393,-0.409381,-0.255308,-3.569663,1.024226,-0.009943,-2.587274,2.320346,3.720155,1.923058,-0.951342,-0.650488,-1.171938,2.526503,-4.171056,5.246707,-5.377279,-0.362354,1.198104,6.404433,4.929743,1.436386,0.120049,1.810783,-3.197694,-1.539112,-3.562278,0.578438,0.097299,1.989754,1.432770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,15.686263,-11.742349,-18.587256,0.770741,20.679988,19.431059,1.269909,-5.663072,-18.041814,14.905343,16.499076,-4.151426,-6.943286,-2.230890,-9.375854,-4.093744,0.935039,-4.104521,1.311996,-4.114335,-5.893210,2.643143,-0.583176,-0.585364,5.947535,-0.705230,2.137562,-2.592799,-7.265226,-1.452370,-0.365752,-0.941695,-5.016188,0.495046,-1.315877,3.063136,-6.576486,-0.112787,2.262566,5.870471,5.178332,1.190152,1.005018,2.614215,-4.342855,-2.115528,-4.038096,1.692400,0.022295,2.039759,1.349730
0,14.702644,-9.732140,-21.165209,11.772768,11.489153,13.883890,-1.296933,-15.852609,-10.656027,15.382970,14.710618,2.234475,9.304977,8.436617,-2.307475,1.736004,8.431571,1.733622,-0.301127,-2.007760,-1.817335,-0.714304,2.109394,3.596233,-3.860881,6.593645,2.688309,-0.288257,-2.848830,-1.273743,7.463113,4.921993,-1.233233,4.822581,-0.772655,3.058628,-6.885608,0.225902,3.764374,6.565262,5.212509,0.919128,1.172911,2.693108,-4.022506,-1.780983,-3.675040,1.856062,0.109955,1.835372,1.351142
0,11.417464,-8.994988,-17.530943,19.322016,17.853326,11.423054,-0.

In [ ]:
gest['col52']= data_2['C52']
gest.head()

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,col51,col52
0,2.981757,0.930167,-6.504309,4.145298,17.914827,11.796062,-3.279976,-10.262140,-12.113334,10.561944,16.700171,-2.743487,4.656042,4.769775,-4.336217,-1.164041,2.694489,0.190694,-1.102917,-1.636381,2.623121,-3.800252,2.892837,0.285310,-0.701675,3.327975,2.191827,3.191380,-1.443529,2.688537,0.482867,-3.332678,-1.434384,4.333419,-2.708510,5.952898,-6.081190,-1.518520,2.555949,5.193041,4.841290,0.944825,-0.324398,2.624692,-3.186598,-2.284424,-3.435495,0.881866,0.091341,2.003784,1.404924,01
0,2.562786,1.175193,-5.584769,4.382689,17.248264,15.854818,-0.778055,-9.443510,-14.761620,10.309589,12.466334,-3.622135,7.322218,3.627075,0.518161,-1.184425,2.747804,-3.839478,-4.190719,-1.342398,-4.194872,-4.409375,1.369324,2.529531,-2.623324,-0.844974,1.695318,2.735177,-1.549703,4.081940,0.879898,5.294059,-3.694276,2.422979,-5.307708,5.512484,-5.911205,-0.167642,1.837931,6.409856,4.369213,0.976204,1.008440,1.968526,-3.486103,-1.683872,-4.133137,0.620888,-0.086144,2.085640,1.412180,01
0,4.421553,1.876080,-7.486937,5.805088,18.657680,11.849551,-2.497996,-11.491260,-14.337524,8.133908,15.353551,-2.411461,1.740058,2.184639,-1.008837,-1.176413,-0.570894,0.165828,-0.643748,0.147326,1.178720,1.860963,3.000123,0.231278,3.299346,3.690817,-4.877524,-0.784119,0.138205,0.823937,5.865086,-0.138524,-4.104444,0.272523,-9.059541,0.037832,-8.928936,-1.120540,2.980539,6.898731,3.088851,-0.470193,2.013683,2.512737,-2.413246,-2.092192,-4.502322,1.458307,-0.124726,1.963769,0.982767,01
0,2.478851,1.608154,-2.689707,5.834392,16.605050,12.700200,-4.093186,-10.220596,-11.758723,9.639325,15.448624,-2.770145,1.617935,5.129490,-4.053044,0.558379,-1.470494,0.423045,-1.049579,-1.543352,0.448407,2.016325,3.017876,-2.825164,-0.181563,0.708133,-4.293064,-0.412121,-3.681796,1.566023,-0.945304,-0.871491,-1.350733,1.559220,-5.684127,2.397597,-7.981382,-0.992044,2.174206,6.666250,3.631261,0.004376,1.089513,2.001774,-2.874913,-1.983045,-4.211922,0.996139,-0.214402,2.144631,1.184338,01
0,1.230209,2.857414,-4.377300,3.648867,19.568313,10.680346,-1.365403,-14.687043,-12.609694,10.118383,15.504839,-2.052668,4.265440,1.457049,-1.615563,0.056012,1.230966,0.048883,-1.306581,-0.910279,0.422393,-0.409381,-0.255308,-3.569663,1.024226,-0.009943,-2.587274,2.320346,3.720155,1.923058,-0.951342,-0.650488,-1.171938,2.526503,-4.171056,5.246707,-5.377279,-0.362354,1.198104,6.404433,4.929743,1.436386,0.120049,1.810783,-3.197694,-1.539112,-3.562278,0.578438,0.097299,1.989754,1.432770,01


In [ ]:
exp_var=s**2/np.sum(s**2)*100  # Explained variance by each eigen value/PC
exp_var
exp_variences = []
for i in range(len(s)):
    exp_variences.append(s[i] / np.sum(s))

#print(np.sum(exp_variences))
#print(exp_variences)
#sum(exp_variences[:51])

In [ ]:
from sklearn.decomposition import PCA 
# Choosing the extent of variance to be covered by the PCs
pca = PCA(n_components=30)

# Transforming the iris data input_columns
pca_data = pca.fit_transform(gest.iloc[:,:51])

# Storing the PCs in the data frame
pca_data_df = pd.DataFrame(pca_data, columns=['col' +str(x) for x in range(1,31)])
	
pca_data_df_with_class = pd.concat([pca_data_df, gest['col52']], axis=0)

print('Explained variance :\n')
print(sum(pca.explained_variance_ratio_))

Explained variance :

0.9743168365054763


In [ ]:
data_3 = pca_data_df.copy()

# Creating a copy of data_2
x = data_3.copy()

# Splitting into train and test data
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state =0)


In [ ]:
# Standardising all the variables 
scalar = StandardScaler()
scalar.fit(train_x)
train_x = scalar.transform(train_x)
test_x = scalar.transform(test_x)

In [ ]:
# Random Forest 
from sklearn.metrics import accuracy_score,confusion_matrix
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier

# Building Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=0)
rf.fit(train_x, train_y)

pred_rf = rf.predict(test_x) 

# Confusion Matrix
print(confusion_matrix(test_y,pred_rf))
# Accuracy Score
print(accuracy_score(test_y,pred_rf))


[[23 25  0  0  0  0  0  0  0  0  0  0  0  0  1  0  1  1  0  0]
 [11 28  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1]
 [ 2  0 31  4  0  0  0  0  4  2  0  1  0  0  0  0  0  1  1  1]
 [ 1  0  3 37  2  0  0  0  1  1  2  4  3  1  0  1  0  0  0  0]
 [ 0  0  0  0 24 11  2  0  0  0  0  0  0  0  1  1  1  1  0  0]
 [ 0  0  0  1  7 35  2  0  0  0  4  1  0  0  1  0  0  0  0  0]
 [ 0  0  0  1  8  5 20 10  1  1  2  2  0  0  1  0  2  0  1  0]
 [ 0  0  0  1  8  9  3 16  0  3  3  6  0  0  2  1  0  0  0  0]
 [ 1  0  3  0  0  0  6  8 14 10  0  0  0  1  0  0  0  0  2  0]
 [ 0  0  1  0  1  0  5  3  9 20  0  1  0  1  0  0  1  0  2  2]
 [ 0  0  0  1  1  2  3  5  1  2 24  4  0  3  0  0  1  2  2  3]
 [ 0  0  0  2  5  1  0  1  5  1  3 18  0  3  0  2  1  2  1  2]
 [ 0  0  5  3  0  0  0  0  3  2  0  3 28  8  0  0  0  0  1  1]
 [ 0  0  5  4  0  0  2  2  1  1  0  0 14 17  1  2  1  0  2  1]
 [ 0  0  0  1  4  2  0  0  0  0  1  0  0  0 25 11  0  0  0  0]
 [ 0  0  0  2  3  1  0  0  0  0  3  0  0  0  7 36  0  0

In [ ]:
# Importing the required libraries 
from sklearn.neighbors import KNeighborsClassifier

# Setting the K value
kclassifier = KNeighborsClassifier(n_neighbors = 3)

# Fitting the values of x and y
kclassifier.fit(train_x, train_y)

# Predicting 
prediction = kclassifier.predict(test_x)

# Confusion Matrix
confusion_matrix = confusion_matrix(prediction, test_y)
print(confusion_matrix)

# Accuracy Score
accuracy_score = accuracy_score(prediction, test_y)
print(accuracy_score)



[[19 18  0  2  0  0  0  0  1  2  2  0  0  0  1  1  2  2  0  1]
 [31 21  0  1  0  0  2  1  1  0  2  0  0  0  0  0  2  2  0  1]
 [ 0  0 28  8  0  0  0  1  3  5  2  3  7  8  0  0  0  0  4  1]
 [ 0  0  4 33  3  4  3  4  3  1  5 13  4  5  2  3  0  1  2  2]
 [ 0  0  1  1 25 15 10 17  0  0  3  2  0  0  5  3  1  2  1  0]
 [ 0  0  0  1  5 21  5  7  1  1  5  0  0  1  0  2  1  1  0  0]
 [ 0  0  1  1  2  5 16  7  5  5  3  4  0  2  1  3  0  0  0  0]
 [ 1  0  0  0  2  3  8  8  2  2  2  3  0  2  0  3  0  0  0  0]
 [ 0  0  3  3  0  0  2  1 17  8  2  1  5  3  0  0  1  0  2  0]
 [ 0  0  5  2  0  0  1  1  6 19  1  1  2  1  0  0  0  0  0  1]
 [ 0  0  0  2  0  2  3  1  0  1 21  6  1  0  3  2  2  1  0  1]
 [ 0  0  2  1  2  0  4  4  0  0  1  9  0  0  0  0  1  0  1  1]
 [ 0  0  1  1  0  0  0  0  2  0  1  0 22 12  0  0  0  0  2  0]
 [ 0  0  2  0  0  0  0  0  3  1  1  1 13 19  0  0  0  0  0  0]
 [ 0  0  0  0  2  1  0  0  0  0  0  0  0  0 20 11  0  2  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0 12 24  0  0

SVM:

In [ ]:
svm_clf = SVC(gamma="scale",C = 10, kernel = 'rbf', degree = 3, random_state=None)
svm_clf.fit(train_x, train_y)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
pred_y = svm_clf.predict(train_x)
print(accuracy_score(train_y, pred_y))

0.9213186813186813
